### Criando a classe dos dados:

In [2]:
import random
import json

In [3]:
class Sentiment:
  NEGATIVE = "NEGATIVE"
  NEUTRAL = "NEUTRAL"
  POSITIVE = "POSITIVE"

In [4]:
class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score
    self.sentiment = self.get_sentiment()

  def get_sentiment(self):
    if self.score <= 2:
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else: #score = 4 ou 5
      return Sentiment.POSITIVE

### Carregando os dados:

In [5]:
file_name = 'data/Books_small.json'

reviews = []

with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].text)
print(reviews[5].score)

Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!
4.0


### Preparando os dados

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

training[0].sentiment

'POSITIVE'

In [8]:
train_x = [x.text for x in training]
train_y = [y.sentiment for y in training]

test_x = [x.text for x in test]
test_y = [y.sentiment for y in test]

print(train_x[0])
print(train_y[0])

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
POSITIVE


##### Vetorização utilizando Bags of Words

&emsp;&emsp;&emsp;&emsp;This is an example.

![alt text](Screenshot_1.jpg "Title")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# transformando os textos em train_x em vetores numéricos
#
train_x_vectors = vectorizer.fit_transform(train_x)
# vectorizer.fit aprende o dicionário com todas as palavras 
# vectorizer.transform transforma a string num vetor utilizando o dicionário aprendido
# vectorizer.fit_transform faz ambas as coisas

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0])

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 17 stored elements and shape (1, 7372)>
  Coords	Values
  (0, 7086)	1
  (0, 1148)	1
  (0, 350)	2
  (0, 1800)	1
  (0, 6595)	1
  (0, 562)	1
  (0, 3054)	1
  (0, 1558)	1
  (0, 6475)	1
  (0, 6593)	1
  (0, 2895)	1
  (0, 7353)	1
  (0, 539)	1
  (0, 1515)	1
  (0, 5197)	1
  (0, 3545)	1
  (0, 2007)	1


#### Classificação

In [19]:
from sklearn.linear_model import LogisticRegression

# criando um classificador
clf_log = LogisticRegression()

# ajustando os vetores
clf_log.fit(train_x_vectors, train_y)

# queremos prever, a partir do nosso modelo, se o seguinte texto, que é positivo,
#  é positivo ou negativo:
print(train_x[0])
print(train_y[0])

# resultado previsto:
print("O texto é: ", clf_log.predict(train_x_vectors[0]))

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
POSITIVE
O texto é:  ['POSITIVE']


#### Avaliação do modelo

In [ ]:
# acurácia média do modelo, ou seja, queremos saber o quão bem, em média,
#  os vetores de treino, em test_x_vectors, predizem os vetores test_y
clf_log.score(test_x_vectors, test_y)

0.8303030303030303

In [ ]:
# Calculando o F1 score, que é uma medida que leva em conta a quantidade de falsos
# positivos, falsos negativos e positivos verdadeiros, é, de certa forma uma medida mais 
# precisa do que a medida score

from sklearn.metrics import f1_score

# recebe o valor real e o valor predito, retorna o quanto o modelo está acertando para
# cada classe que queremos prever
#
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, 
         labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

array([0.91370558, 0.12244898, 0.1       ])

Notamos que o nosso modelo tem uma boa capacidade para prever valores positivos, porém peca em prever valores negativos e neutros.

In [29]:
print('Quantidade de valores POSITIVOS no conjunto de treino: ',
      train_y.count(Sentiment.POSITIVE))

print('Quantidade de valores NEGATIVOS no conjunto de treino: ',
      train_y.count(Sentiment.NEGATIVE))

Quantidade de valores POSITIVOS no conjunto de treino:  552
Quantidade de valores NEGATIVOS no conjunto de treino:  47


Notamos que a quantidade de valores POSITIVOS no nosso conjunto de treinamento é muito maior que a quantidade de valores NEGATIVOS, por esse motivo, a capacidade do modelo de prever valores negativos é fortemente prejudicada. Para resolver isso, aumentaremos o banco de dados.